In [1]:
from __future__ import print_function, division

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import qgrid

import numpy as np
from numpy import diff
import math

import pandas as pd
import xarray as xr

import climlab
from climlab.solar.insolation import daily_insolation
from climlab.radiation import DailyInsolation
from climlab.radiation import FixedInsolation
from climlab.process import TimeDependentProcess
from climlab.utils import heat_capacity
from matplotlib.pyplot import cm
from matplotlib.lines import Line2D
from matplotlib.legend import Legend

from IPython.display import HTML


import scipy as scp
from attrdict import AttrDict
from datetime import datetime

import dask.dataframe as dd

import warnings

import util
import ram_model
import plotting

/opt/anaconda3/envs/conda_env/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Import Observational Data

Read data and concatenate into one dataset.

level 0 = Surface

level 110 = TOA

(our data is in the order TOA-->Surface)

In [3]:
ds = util.load_soundings()
util.add_monthly_insolation(ds)
ds = util.adjust_lev(ds, nlevels = 95)
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
co2_lev = ds.CO2_list.values
run_name = 'low7_albedo'# 'normal'

#create an option to add one degree to the surface temperature in order to calculate an energy budget
TS_plus_one = False

if TS_plus_one == True:
    ds.isel(level = -1)['Temperature']+= 1
    run_name = TS_plus_one

Dropped levels [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 14, 15, 17, 18, 20, 22, 24, 26]


# Climlab

All tendencies must be in K/sec (see Climlab energy_budget documentation and self.heating_rate attribute): https://climlab.readthedocs.io/en/latest/_modules/climlab/process/energy_budget.html#EnergyBudget

In [13]:
timestep = 300 #seconds

############# FIND OUR AVERAGE KAPPA AND ADVECTION ###############

#### Find the surface diffk average over 12 months at CO2 = .00038 ####
mean_diffk_dict = {}
mean_diffk_dict = ram_model.fill_ensemble(ds, mean_diffk_dict, timestep, 
                                                     advection_on = True, turbulence_on = True, 
                                                     advection = None, 
                                                     surface_diffk = None)
surface_diffk_average = ram_model.annual_mean_sfc_diffk(ds, mean_diffk_dict)
print(surface_diffk_average)

#### Find our initial advection #### 
adv_dict = {}
adv_dict = ram_model.fill_ensemble(ds, adv_dict, timestep, 
                                   advection_on = True, turbulence_on = True, 
                                   advection = None, 
                                   surface_diffk = surface_diffk_average)
advection_380_monthly = {}
for m in months:
    advection_380_monthly[m] = adv_dict[0.00038][m].subprocess.Advection.forcing_tendencies['Tatm']


[0.36798833]


In [14]:
mean_diffk_dict

{0.0: {'January': <climlab.process.time_dependent_process.TimeDependentProcess at 0x7f9208f40dd0>,
  'February': <climlab.process.time_dependent_process.TimeDependentProcess at 0x7f9209fecfd0>,
  'March': <climlab.process.time_dependent_process.TimeDependentProcess at 0x7f9209ff9e50>,
  'April': <climlab.process.time_dependent_process.TimeDependentProcess at 0x7f9209fdde10>,
  'May': <climlab.process.time_dependent_process.TimeDependentProcess at 0x7f9209fdda90>,
  'June': <climlab.process.time_dependent_process.TimeDependentProcess at 0x7f9209fe8550>,
  'July': <climlab.process.time_dependent_process.TimeDependentProcess at 0x7f9209fe8750>,
  'August': <climlab.process.time_dependent_process.TimeDependentProcess at 0x7f9209fe8150>,
  'September': <climlab.process.time_dependent_process.TimeDependentProcess at 0x7f9209fe8950>,
  'October': <climlab.process.time_dependent_process.TimeDependentProcess at 0x7f9209fe8f90>,
  'November': <climlab.process.time_dependent_process.TimeDependent

## Create our Models
Use our initial advection and surface diffk to run the model

Run different versions of the model in the case that we want to examine just one component (out of Radiative, Advective and Turbulent)

In [30]:
########### RADIATIVE ADVECTIVE TURBULENT #############
rat_dict = {}
rat_dict = ram_model.fill_ensemble(ds, rat_dict, timestep, 
                                   advection_on = True, turbulence_on = True, 
                                   advection = advection_380_monthly,
                                   surface_diffk = surface_diffk_average)



In [31]:
########### RADIATIVE ADVECTIVE #############
ra_dict = {}
ra_dict = ram_model.fill_ensemble(ds, ra_dict, timestep, 
                                   advection_on = True, turbulence_on = False, 
                                   advection = advection_380_monthly,
                                   surface_diffk = surface_diffk_average)



In [32]:
########### RADIATIVE TURBULENT #############
rt_dict = {}
rt_dict = ram_model.fill_ensemble(ds, rt_dict, timestep, 
                                   advection_on = False, turbulence_on = True, 
                                   advection = None,
                                   surface_diffk = surface_diffk_average)

In [33]:
########### RADIATIVE #############
r_dict = {}
r_dict = ram_model.fill_ensemble(ds, r_dict, timestep, 
                                   advection_on = False, turbulence_on = False, 
                                   advection = None,
                                   surface_diffk = surface_diffk_average)

# Output (initial and over time)

In [34]:
month_ds_dict = {} #Empty dictionary to add values into
steps = 43201 #choose number of timesteps
fields_dict = {'TdotLW_clr':'z', 'LW_flux_up_clr':'z_bounds', 
               'LW_flux_down_clr':'z_bounds', 'LW_flux_net_clr':'z_bounds', 
               'TdotSW_clr':'z', 'SW_flux_up_clr':'z_bounds', 
               'SW_flux_down_clr':'z_bounds', 'SW_flux_net_clr':'z_bounds'} #fields from climlab output and their corresponding grid levels
model_dict = {'rat':rat_dict,
             #'rt':rt_dict,
             #'ra':ra_dict,
             #'r':r_dict
             }

### Initial Timestep

In [35]:
#original
ex_dict = rat_dict.copy()
save_freq = 288 #frequency to save out the data
for month in months:
    #create a dataset for each month
    month_ds_dict[month] = xr.Dataset(data_vars = {
                                    'model' : ('model', ['rat','rt','ra','r']),
                                    'co2_lev': ('co2_lev', ds['CO2_list'].values), 
                                    'time': ('time', np.arange(0,((steps+1)*timestep),save_freq*timestep)),
                                    'lev': ('lev', ex_dict[.00038][month].lev),
                                    'lev_full': ('lev_full', np.append(ex_dict[.00038][month].lev, 
                                                                       ex_dict[.00038][month].lev[-1]+(ex_dict[.00038][month].lev[-1]-ex_dict[.00038][month].lev[-2]))),
                                    'lev_bounds': ('lev_bounds', ex_dict[.00038][month].lev_bounds)
                                    })
    #create a data-array for the pressure levels at the center of each grid box
    lev_da = xr.DataArray(
                        data = np.zeros((month_ds_dict[month].dims['model'],month_ds_dict[month].dims['co2_lev'],month_ds_dict[month].dims['lev'],month_ds_dict[month].dims['time'])), 
                        coords = ((month_ds_dict[month].coords['model'], month_ds_dict[month].coords['co2_lev'], month_ds_dict[month].coords['lev'], month_ds_dict[month].coords['time'])),
                        dims = ('model','co2_lev', 'lev','time')
                    )

    #create a data-array for the pressure levels at the edges of each grid box
    lev_bounds_da = xr.DataArray(
                        data = np.zeros((month_ds_dict[month].dims['model'], month_ds_dict[month].dims['co2_lev'],month_ds_dict[month].dims['lev_bounds'],month_ds_dict[month].dims['time'])), 
                        coords = ((month_ds_dict[month].coords['model'], month_ds_dict[month].coords['co2_lev'], month_ds_dict[month].coords['lev_bounds'], month_ds_dict[month].coords['time'])),
                        dims = ('model','co2_lev', 'lev_bounds','time')
                    )
    #create a data-array for the pressure levels at the center of each grid box, including the surface
    lev_full_da = xr.DataArray(
                        data = np.zeros((month_ds_dict[month].dims['model'], month_ds_dict[month].dims['co2_lev'],month_ds_dict[month].dims['lev_full'],month_ds_dict[month].dims['time'])), 
                        coords = ((month_ds_dict[month].coords['model'], month_ds_dict[month].coords['co2_lev'], month_ds_dict[month].coords['lev_full'], month_ds_dict[month].coords['time'])),
                        dims = ('model','co2_lev', 'lev_full','time')
                    )
    #assign our outputs to a given array size
    month_ds_dict[month]['T'] = lev_full_da.copy()
    month_ds_dict[month]['turb_hr'] = lev_full_da.copy()
    month_ds_dict[month]['turbulent_flux'] = lev_bounds_da.copy()
    month_ds_dict[month]['diffk'] = lev_bounds_da.copy()
    month_ds_dict[month]['theta_init'] = lev_da.copy()
    month_ds_dict[month]['theta'] = lev_da.copy()
    month_ds_dict[month]['turb_atm_hr'] = lev_da.copy()
    month_ds_dict[month]['advection'] = lev_full_da.copy() 
    month_ds_dict[month]['heat_capacity'] = lev_da.copy()
    
    for var in fields_dict.keys():
        if fields_dict[var] == 'z':
            month_ds_dict[month][var] = lev_da.copy()
        if fields_dict[var] == 'z_bounds':
            month_ds_dict[month][var] = lev_bounds_da.copy()
            
    for co2 in ex_dict.keys():
        for dict_name in model_dict.keys():
            #create z coords
            month_ds_dict[month] = month_ds_dict[month].assign_coords(z = ex_dict[co2][month].z)
            month_ds_dict[month] = month_ds_dict[month].assign_coords(z_bounds = ex_dict[co2][month].z_bounds)
            month_ds_dict[month] = month_ds_dict[month].assign_coords(z_full = np.append(ex_dict[co2][month].z, -0.5))
            
            month_ds_dict[month]['T'].loc[dict(co2_lev = co2, time = 0, model = dict_name)] = np.append(np.array(model_dict[dict_name][co2][month].state['Tatm']),
                                                         np.array(model_dict[dict_name][co2][month].state['Ts']))
            month_ds_dict[month]['heat_capacity'].loc[dict(co2_lev = co2, time = 0, model = dict_name)] = model_dict[dict_name][co2][month].domains['Tatm'].heat_capacity

            if 'a' in dict_name:
                month_ds_dict[month]['advection'].loc[dict(co2_lev = co2, time = 0, model = dict_name)] = np.append(np.array(model_dict[dict_name][co2][month].subprocess.Advection.forcing_tendencies['Tatm']),
                                                         np.array(model_dict[dict_name][co2][month].subprocess.Advection.forcing_tendencies['Ts']))
            
            if 't' in dict_name:
                month_ds_dict[month]['turb_hr'].loc[dict(co2_lev = co2, time = 0, model = dict_name)] = np.append(np.array(model_dict[dict_name][co2][month].diagnostics['turb_atm_hr']),
                                                             np.array(model_dict[dict_name][co2][month].diagnostics['turb_ground_hr']))
                month_ds_dict[month]['turbulent_flux'].loc[dict(co2_lev = co2, time = 0, model = dict_name)] = (np.array(model_dict[dict_name][co2][month].diagnostics['atm_turbulent_flux']))
                month_ds_dict[month]['diffk'].loc[dict(co2_lev = co2, time = 0, model = dict_name)] = np.append(np.array(model_dict[dict_name][co2][month].diagnostics['atm_diffk']),
                                                             np.array(model_dict[dict_name][co2][month].diagnostics['surface_diffk']))
                month_ds_dict[month]['theta_init'].loc[dict(co2_lev = co2, time = 0, model = dict_name)] = model_dict[dict_name][co2][month].diagnostics['theta_init']
                month_ds_dict[month]['theta'].loc[dict(co2_lev = co2, time = 0, model = dict_name)] = model_dict[dict_name][co2][month].diagnostics['theta']
                month_ds_dict[month]['turb_atm_hr'].loc[dict(co2_lev = co2, time = 0, model = dict_name)] = model_dict[dict_name][co2][month].diagnostics['turb_atm_hr']

            for var in fields_dict.keys():
                month_ds_dict[month][var].loc[dict(co2_lev = co2, time = 0, model = dict_name)] = model_dict[dict_name][co2][month].diagnostics[var].to_xarray().data


### Timestepping

In [12]:
#timestepper
for month in months:
    for dict_name in model_dict.keys():
        for co2 in rat_dict.keys():
            tmp_model = model_dict[dict_name][co2][month]
            for i in range(1,steps):
                #step the model forward
                tmp_model.step_forward()
                if i*timestep in month_ds_dict[month]['time']:
                    loc_dict = dict(co2_lev = co2, time = (i*timestep), model = dict_name)
                    #assign temperature
                    month_ds_dict[month]['T'].loc[loc_dict] = np.append(np.array(tmp_model.state['Tatm']),
                                                                 np.array(tmp_model.state['Ts']))
                    #assign heat capacity
                    month_ds_dict[month]['heat_capacity'].loc[loc_dict] = tmp_model.domains['Tatm'].heat_capacity
                    #assign the variables in the 'field dict' which are typical outputs from climlab
                    for var in fields_dict.keys():

                        month_ds_dict[month][var].loc[loc_dict] = tmp_model.diagnostics[var].to_xarray().data


                    #assign the variables in the 'advection' field
                    if 'a' in dict_name:

                        month_ds_dict[month]['advection'].loc[loc_dict] = np.append(np.array(tmp_model.subprocess.Advection.forcing_tendencies['Tatm']),
                                                                 np.array(tmp_model.subprocess.Advection.forcing_tendencies['Ts']))
                    #assign the variables in the 'turbulence field'
                    if 't' in dict_name:

                        # WHY ONLY .to_xarray().data on some of the diagnostics? What is the point of this?
                        month_ds_dict[month]['turb_hr'].loc[loc_dict] = np.append(np.array(tmp_model.diagnostics['turb_atm_hr']),
                                                                     np.array(tmp_model.diagnostics['turb_ground_hr']))
                        month_ds_dict[month]['turbulent_flux'].loc[loc_dict] = np.array(tmp_model.diagnostics['atm_turbulent_flux'])#,
                                                                     #np.array(tmp_model.diagnostics['sfc_turbulent_flux']))
                        month_ds_dict[month]['diffk'].loc[loc_dict] = np.append(np.array(tmp_model.diagnostics['atm_diffk']),
                                                                     np.array(tmp_model.diagnostics['surface_diffk']))
                        month_ds_dict[month]['theta_init'].loc[loc_dict] = tmp_model.diagnostics['theta_init']
                        month_ds_dict[month]['theta'].loc[loc_dict] = tmp_model.diagnostics['theta']
                        month_ds_dict[month]['turb_atm_hr'].loc[loc_dict] = tmp_model.diagnostics['turb_atm_hr']



month_ds_dict = {} 

for month in months: 
    month_ds_dict[month] = xr.open_dataset(f'../output/{month}_300s_43200ts_ds')
times = month_ds_dict[month].time.values


## Calculate the Surface CO2 Effect, Effective Bandwidth

In [13]:
#calculate surface CO2 Effect
for month in months:
    month_ds_dict[month]['sfc_co2_effect'] = (month_ds_dict[month].sel(time = month_ds_dict[month].time.values[-1], model = 'rat').isel(lev_bounds = -1)['LW_flux_down_clr'] -
         month_ds_dict[month].sel(time = month_ds_dict[month].time.values[-1], model = 'rat', co2_lev = 0).isel(lev_bounds = -1)['LW_flux_down_clr'])

#calculate planck function and effective bandwidth    
h = 6.626E-34 #J*S
n = 667 #cm
kb = 1.38E-23 #J/k
c = 2.998E10 #cm/s
for month in months:
    #planck function
    Ta = month_ds_dict[month]['T'].sel(model = 'rat', time = 0).isel(lev_full = -1)
    month_ds_dict[month]['planck_function'] = ((2*h*c**2*n**3)/(np.exp((h*c*n)/(kb*Ta)) - 2))*1E4*np.pi
    #effective bandwidth
    month_ds_dict[month]['eff_bandwidth'] = month_ds_dict[month]['sfc_co2_effect']/month_ds_dict[month]['planck_function']
    
#effective bandwidth calculated as in Jeevanjee et al (2020, eq. 7)
l = 10.4 #cm-1
C0 = 0.25
C = month_ds_dict['January']['co2_lev'][1:].values*1e6
eff_bandwidth_est = 2*l*np.log(C/C0)

for month in months:
    month_ds_dict[month]['sfc_co2_estimate'] = eff_bandwidth_est*month_ds_dict[month]['planck_function'][1:]
    month_ds_dict[month]['eff_bandwidth_est'] = eff_bandwidth_est

# Save Output

In [36]:
for month in months:
    month_ds_dict[month].to_netcdf(f'../output/{month}_{run_name}_ds')